In [97]:
import json
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import f1_score

from sys import path

path.append('../utils/')

from utils import get_datasets

from utils import load_preds

from scipy.stats import t as table_t

In [98]:
DATASETS = ["20ng", "acm", "webkb", "reut"]
DATASETS = ["acm", "webkb", "20ng"]

In [99]:
pd_datasets = get_datasets(DATASETS)

In [100]:
MF_SET = [
    "probas-based"
]
MF_SET.sort()
mf_sufix = '_'.join(MF_SET)

CLFS1 = [
["bert", "normal_probas"],
["xlnet", "normal_probas"],
["ktmk", "normal_probas"],
["ktr", "normal_probas"],
["lstmk", "normal_probas"],
["lstr", "normal_probas"],
["ltr", "normal_probas"]
]

CLFS2 = [
["bert", "temperature_scaling"],
["xlnet", "temperature_scaling"],
["ktmk", "isotonic"],
["ktr", "isotonic"],
["lstmk", "isotonic"],
["lstr", "isotonic"],
["ltr", "isotonic"]
]

posfix = '/'.join(sorted([ f"{c[0]}_{c[1]}" for c in CLFS2 ]))

st1 = f"calibrated/{posfix}"

posfix = '/'.join(sorted([ f"{c[0]}_{c[1]}" for c in CLFS2 ]))

"""
st2s = [
    f"error_detection/zero_train_True/{alfa}/{posfix}/centroids-ratios_dist_neigborhood_probas_probas-based"
    for alfa in [0.5]
]
st2s = [
    f"upper_bound/zero_train_True/{alfa}/{posfix}"
    for alfa in [0.5]
]
"""

st2s = [ posfix ]
#st2 = f"normal_probas/{posfix}"
#st2s = ["calibrated"]

fold_values = {}
for dataset in DATASETS:
    fold_values[dataset] = {}
    for st2 in st2s:
        fold_values[dataset][st2] = []
        diffs = []
        alpha = 0.05 / 2
        for fold in np.arange(10):

            ppath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/logistic_regression/{st1}/fold_{fold}/scoring.json"
            with open(ppath, 'r') as fd:
                st1_macro = json.load(fd)["f1_macro"]
            
            #mpath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/attention/dual_cross_attention/use_mf_False/batch_64/{st2}/fold_{fold}/scoring.json"
            mpath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/logistic_regression/hard_docs_upper_bound/{st2}/{mf_sufix}/fold_{fold}/scoring.json"
            with open(mpath, 'r') as fd:
                st2_macro = json.load(fd)["f1_macro"]
            
            diffs.append(st2_macro - st1_macro)
            fold_values[dataset][st2].append([st1_macro, st2_macro])
        #    print(st2_macro, st1_macro, st2_macro - st1_macro)
            
        mean = np.mean(diffs)
        #print(mean)
        std = np.std(diffs) / np.sqrt(len(diffs))

        t = table_t.ppf(1 - alpha, df=9)
        ground = np.round(mean - t * std, decimals=4)
        ceiling = np.round(mean + t * std, decimals=4)

        print(f"{dataset.upper()} - ({ground} - {ceiling})")
        #print("*"*100)

ACM - (0.0145 - 0.0245)
WEBKB - (0.0155 - 0.0367)
20NG - (-0.0016 - 0.0048)


In [101]:
for dataset in DATASETS:
    for st2 in st2s:
        df = pd.DataFrame(fold_values[dataset][st2], columns=["st1", "st2"])
        mrep = np.round(np.mean(df.st1.values) * 100, decimals=2)
        prob = np.round(np.mean(df.st2.values) * 100, decimals=2)
        print(f"{dataset.upper()} - st1: {mrep} st2: {prob}")
        print(st1," -- ", st2)

ACM - st1: 73.74 st2: 75.68
calibrated/bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling  --  bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling
WEBKB - st1: 83.13 st2: 85.74
calibrated/bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling  --  bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling
20NG - st1: 92.18 st2: 92.34
calibrated/bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling  --  bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling


In [102]:
# Dual MP

# WEBKB - (0.0158 - 0.0326)
# WEBKB - st1: 82.91 st2: 85.33
# WEBKB - (-0.0391 - -0.0103)
# WEBKB - st1: 82.91 st2: 80.44